In [16]:
# lista de bibliotecas
import importlib
import subprocess

# Lista de bibliotecas que você precisa verificar
bibliotecas = ['ftplib', 'io', 'py7zr', 'pandas', 'os', 'datetime', 'unidecode', 'pyodbc', 'sqlite3']

# Verifica se cada biblioteca está instalada e, se necessário, instala
for biblioteca in bibliotecas:
    try:
        importlib.import_module(biblioteca)
        print(f'{biblioteca} já está instalada.')
    except ImportError:
        print(f'{biblioteca} não está instalada. Instalando...')
        subprocess.check_call(['pip', 'install', biblioteca])
        print(f'{biblioteca} instalada com sucesso.')        

ftplib já está instalada.
io já está instalada.
py7zr já está instalada.
pandas já está instalada.
os já está instalada.
datetime já está instalada.
unidecode já está instalada.
pyodbc já está instalada.
sqlite3 já está instalada.


In [17]:
# lista de bibliotecas
from ftplib import FTP
from py7zr import SevenZipFile
import pandas as pd
import os
import datetime as dt
from unidecode import unidecode
import pyodbc
import io
import sqlite3

In [18]:
# importando as bases
cnae = pd.read_excel('./bases/cnae_classe.xlsx')
regional = pd.read_excel('./bases/base_regional.xlsx')
cbo = pd.read_excel('./bases/estrutura_cbo.xlsx')

In [19]:
# função para remover acentuação dos dataframes
def remover_acentuacao_titulos(df):
    df = df.rename(columns=lambda x: unidecode(x) if isinstance(x, str) else x)
    return df

In [20]:
dataframes = {}                    # Dicionário para armazenar os DataFrames
data = dt.datetime.now()           # data atual
ano = int(data.strftime("%Y"))     #  pegando apenas o ano como inteiro
meses = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]       # array de meses

In [21]:
url = 'ftp.mtps.gov.br'  # URL do servidor FTP
diretorio = f'/pdet/microdados/NOVO CAGED/{ano}'  # Diretório desejado
ftp = FTP(url)
ftp.login()  # Realiza o login no servidor FTP
ftp.cwd(diretorio)  # Navega para o diretório desejado
diretorios = ftp.nlst()  # Obtém a lista de diretórios no diretório atual
diretorios_ordenados = sorted(diretorios)  # Ordena a lista de diretórios em ordem crescente
ultimo_diretorio = diretorios_ordenados[0]  # Seleciona o último diretório
ftp.cwd(ultimo_diretorio) # Acessando a pasta atual
arquivos = ftp.nlst() # listando os arquivos

In [22]:
for arquivo in arquivos:    # listando os arquivos da ultima pasta
    print(arquivo)

CAGEDEXC202301.7z
CAGEDFOR202301.7z
CAGEDMOV202301.7z


In [23]:
# percorrendo cada arquivo
for arquivo in arquivos:
    if arquivo.endswith('.7z'):  # Verifica se o arquivo é do tipo 7z
        with io.BytesIO() as mem_file:
            ftp.retrbinary('RETR ' + arquivo, mem_file.write)  # Baixa o arquivo 7z em memória
            mem_file.seek(0)  # Reposiciona o ponteiro do arquivo no início            
            with SevenZipFile(mem_file, mode='r') as archive:
                archive.extractall()  # Extrai o conteúdo do arquivo 7z                
            nome_arquivo_txt = os.path.splitext(arquivo)[0] + '.txt'  # Obtém o nome do arquivo txt
            with open(nome_arquivo_txt, 'r', encoding='UTF-8') as file:
                # Processa o arquivo txt conforme necessário, por exemplo, lendo como um DataFrame:
                df = pd.read_csv(file, sep=';')                
            os.remove(nome_arquivo_txt)  # Remove o arquivo txt extraído do 7z            
            # Armazena o DataFrame no dicionário
            nome_variavel = os.path.splitext(arquivo)[0].replace('CAGEDEST_', '')  # Define o nome da variável
            dataframes[nome_variavel] = df
ftp.quit()  # Encerra a conexão com o servidor FTP

'221 Goodbye.'

In [24]:
# Obter a lista de dataframes
lista_dataframes = list(dataframes.values())

# Atribuir cada dataframe a uma variável pelo índice correspondente
cagedxc = lista_dataframes[0]
cagedfor = lista_dataframes[1]
cagedmov = lista_dataframes[2]

# remover acentuação dos dataframes
cagedmov = remover_acentuacao_titulos(cagedmov)
cagedxc = remover_acentuacao_titulos(cagedxc)
cagedfor = remover_acentuacao_titulos(cagedfor)

In [25]:
dataframes = [cagedmov, cagedfor, cagedxc] # lista com os dataframes
colunas_numericas = ['idade']              # coluna idade
colunas_texto = ['valorsalariofixo', 'salario', 'horascontratuais'] # coluna valorsalariofixo', 'salario', 'horascontratuais'

for df in dataframes:        # percorrendo os dataframes
    for coluna in colunas_numericas:          # percorrendo a coluna idade
        df[coluna].fillna(0, inplace=True)    # substituindo os valores nulos por 0
        df[coluna] = df[coluna].astype('int64')   # convertendo coluna idade de float para int
        df['mesano_referencia'] = ultimo_diretorio[-2:] + ultimo_diretorio[:-2]  # Pegando mês de referência do FTP
    
    for coluna in colunas_texto: # percorrendo as colunas tipo texto
        df[coluna].fillna('', inplace=True) # # substituindo os valores nulos por vazios ''

In [26]:
# substituindo o 0 por campo vazios da coluna idade para seguir o padrão as outras colunas
cagedmov.idade = cagedmov.idade.replace(0, '') 
cagedxc.idade = cagedxc.idade.replace(0, '')
cagedfor.idade = cagedfor.idade.replace(0, '')

In [75]:
# Conectando ao banco de dados
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=MEU SERVIDOR;'
                      'Database=EXEMPLO;'
                      'encondig=latin1;'
                      'Trusted_Connection=True;')

In [76]:
# Nome das tabelas no banco de dados
tb_cagedmov = 'cagedmovDEV'
tb_cagedexc = 'cagedexcDEV'
tb_cagedfor = 'cagedforDEV'

In [77]:
# Criando um cursor para executar as operações no banco de dados
cursor = conn.cursor()

In [78]:
# Limpando as tabelas
tabelas = [tb_cagedmov, tb_cagedexc, tb_cagedfor]

# Itera sobre cada tabela na lista
for tabela in tabelas:
    cursor.execute(f'TRUNCATE TABLE {tabela}')

In [79]:
# Inserindo os dados do DataFrame CAGEDEXC na tabela CAGEDEXC
for index, row in cagedxc.iterrows():
    cursor.execute(f"INSERT INTO {tb_cagedexc} \
    (competenciamov, regiao, uf, municipio, secao, subclasse,\
    saldomovimentacao, cbo2002ocupacao, categoria, graudeinstrucao,\
    idade, horascontratuais, racacor, sexo, tipoempregador,\
    tipoestabelecimento, tipomovimentacao, tipodedeficiencia,\
    indtrabintermitente, indtrabparcial, salario, tamestabjan,\
    indicadoraprendiz, origemdainformacao, competenciadec, competenciaexc,\
    indicadordeexclusao, indicadordeforadoprazo, unidadesalariocodigo, valorsalariofixo,\
    mesano_referencia) \
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
    (row['competenciamov'], row['regiao'], row['uf'], row['municipio'], row['secao'], row['subclasse'], \
    row['saldomovimentacao'], row['cbo2002ocupacao'], row['categoria'], row['graudeinstrucao'], \
    row['idade'], str(row['horascontratuais']), row['racacor'], row['sexo'], row['tipoempregador'], \
    row['tipoestabelecimento'], row['tipomovimentacao'], row['tipodedeficiencia'], row['indtrabintermitente'], \
    row['indtrabparcial'], str(row['salario']), row['tamestabjan'], row['indicadoraprendiz'], row['origemdainformacao'], \
    row['competenciadec'], row['competenciaexc'], row['indicadordeexclusao'], row['indicadordeforadoprazo'], \
    row['unidadesalariocodigo'], str(row['valorsalariofixo']), row['mesano_referencia']))


In [80]:
# Inserindo os dados do DataFrame CAGEDFOR na tabela CAGEDFOR
for index, row in cagedfor.iterrows():
    cursor.execute(f"INSERT INTO {tb_cagedfor} \
    (competenciamov, regiao, uf, municipio, secao, subclasse,\
    saldomovimentacao, cbo2002ocupacao, categoria, graudeinstrucao,\
    idade, horascontratuais, racacor, sexo, tipoempregador,\
    tipoestabelecimento, tipomovimentacao, tipodedeficiencia,\
    indtrabintermitente, indtrabparcial, salario, tamestabjan,\
    indicadoraprendiz, origemdainformacao, competenciadec,\
    indicadordeforadoprazo, unidadesalariocodigo, valorsalariofixo,\
    mesano_referencia) \
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
    (row['competenciamov'], row['regiao'], row['uf'], row['municipio'], row['secao'], row['subclasse'], \
    row['saldomovimentacao'], row['cbo2002ocupacao'], row['categoria'], row['graudeinstrucao'], \
    row['idade'], str(row['horascontratuais']), row['racacor'], row['sexo'], row['tipoempregador'], \
    row['tipoestabelecimento'], row['tipomovimentacao'], row['tipodedeficiencia'], row['indtrabintermitente'], \
    row['indtrabparcial'], str(row['salario']), row['tamestabjan'], row['indicadoraprendiz'], row['origemdainformacao'], \
    row['competenciadec'], row['indicadordeforadoprazo'], row['unidadesalariocodigo'], str(row['valorsalariofixo']), \
    row['mesano_referencia']))

In [81]:
# Inserindo os dados do DataFrame CAGEDMOV na tabela CAGEDMOV
for index, row in cagedmov.iterrows():
    cursor.execute(f"INSERT INTO {tb_cagedmov} \
    (competenciamov, regiao, uf, municipio, secao, subclasse,\
    saldomovimentacao, cbo2002ocupacao, categoria, graudeinstrucao,\
    idade, horascontratuais, racacor, sexo, tipoempregador,\
    tipoestabelecimento, tipomovimentacao, tipodedeficiencia,\
    indtrabintermitente, indtrabparcial, salario, tamestabjan,\
    indicadoraprendiz, origemdainformacao, competenciadec,\
    indicadordeforadoprazo, unidadesalariocodigo, valorsalariofixo,\
    mesano_referencia) \
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
    (row['competenciamov'], row['regiao'], row['uf'], row['municipio'], row['secao'], row['subclasse'], \
    row['saldomovimentacao'], row['cbo2002ocupacao'], row['categoria'], row['graudeinstrucao'], \
    row['idade'], str(row['horascontratuais']), row['racacor'], row['sexo'], row['tipoempregador'], \
    row['tipoestabelecimento'], row['tipomovimentacao'], row['tipodedeficiencia'], row['indtrabintermitente'], \
    row['indtrabparcial'], str(row['salario']), row['tamestabjan'], row['indicadoraprendiz'], row['origemdainformacao'], \
    row['competenciadec'], row['indicadordeforadoprazo'], row['unidadesalariocodigo'], str(row['valorsalariofixo']), \
    row['mesano_referencia']))

In [82]:
conn.commit() # Confirmando as alterações no banco de dados
conn.close() # Fechando a conexão com o banco de dados
print("dados injetados com sucesso no banco de dados.") # comentário final para o power automate capturar

dados injetados com sucesso no banco de dados.


In [28]:
#### EXPORTANDO EXCEL FILTRADO ###

# Criar uma conexão com um banco de dados em memória
conn = sqlite3.connect(':memory:')

# Carregar os dataframes como tabelas SQL
cbo.to_sql('CBO_ESTRUTURA', conn, index=False)
cnae.to_sql('CNAE_CLASSES', conn, index=False)
regional.to_sql('BASE_REGIONAL', conn, index=False)
cagedmov.to_sql('cagedmovDEV', conn, index=False)

3665155

In [29]:
# Executar a consulta SQL
try:
    query = '''
    SELECT 
        cgmov.mesano_referencia,
        CASE cgmov.sexo
            WHEN 1 THEN 'Homem'
            WHEN 3 THEN 'Mulher'
            WHEN 9 THEN 'Nao identificado'
        END AS [SEXO],
        CASE
            WHEN idade > 0 AND idade <= 17 THEN 'até 17 anos'
            WHEN idade > 17 AND idade <= 24 THEN 'de 18 a 24 anos'
            WHEN idade > 24 AND idade <= 29 THEN 'de 25 a 29 anos'
            WHEN idade > 29 AND idade <= 39 THEN 'de 30 a 39 anos'
            WHEN idade > 39 AND idade <= 49 THEN 'de 40 a 49 anos'
            WHEN idade > 49 AND idade <= 64 THEN 'de 50 a 64 anos'
            WHEN idade >= 65 THEN 'de 65 anos ou mais'
            ELSE 'sem idade'
        END AS Faixa_Etaria,  
        CASE cgmov.saldomovimentacao
            WHEN 1 THEN 'Admissão'
            WHEN -1 THEN 'Demitidos'
        END AS [ADMITIDOS  / DEMITIDOS],
        baserg.COD_UF,
        baserg.SIGLA_UF,
        baserg.NOME_UF,
        baserg.COD_REGIAO,
        baserg.NOME_REGIAO,
        CASE
            WHEN graudeinstrucao = '1' THEN 'Analfabetos'
            WHEN graudeinstrucao IN ('2', '3', '4') THEN 'Fundamental incompleto'
            WHEN graudeinstrucao = '5' THEN 'Fundamental completo'
            WHEN graudeinstrucao = '6' THEN 'Médio incompleto'
            WHEN graudeinstrucao = '7' THEN 'Médio completo'
            WHEN graudeinstrucao = '8' THEN 'Superior incompleto'
            WHEN graudeinstrucao IN ('9', '10', '11', '80') THEN 'Superior completo'
            ELSE 'Não identificado'
        END AS [Grau de Instrucao],
        cnae.COD_CNAE_CLASSE,
        cnae.DESC_CNAE_CLASSE,
        cbo.COD_CBO_GRUPO,
        cbo.DESC_CBO_GRUPO,
        cbo.COD_CBO_PSUB,
        cbo.DESC_CBO_PSUB,
        cbo.COD_CBO_SUBGRUPO,
        cbo.DESC_CBO_SUBGRUPO,
        cbo.COD_CBO_FAMILIA,
        cbo.DESC_CBO_FAMILIA,
        cbo.COD_CBO_OCUP,
        cbo.DESC_CBO_OCUP
    FROM cagedmovDEV cgmov
    INNER JOIN CBO_ESTRUTURA cbo ON cbo.COD_CBO_OCUP = cgmov.cbo2002ocupacao
    INNER JOIN CNAE_CLASSES cnae ON cgmov.subclasse = cnae.COD_CNAE_SUBCLASSE
    INNER JOIN BASE_REGIONAL baserg ON baserg.COD_MUNIC = cgmov.municipio
    WHERE cnae.COD_CNAE_CLASSE IN 
    ('19217', '19225', '20215', '20312', '46818', '46826', '47318', '28518', '30113', '30121', '33171', 
    '6000', '9106', '52311', '52320', '52397', '50114', '50122', '50211', '50301')
    '''
    result = pd.read_sql_query(query, conn)
    conn.close()
    print('executou com sucesso.')
except OSError as e:                                               
    print(f"Erro: {str(e)}")

executou com sucesso.


In [30]:
result.to_excel('./bases/df_final.xlsx', index=False)
print('excel exportado')

excel exportado


In [31]:
result

,mesano_referencia,SEXO,Faixa_Etaria,ADMITIDOS / DEMITIDOS,COD_UF,SIGLA_UF,NOME_UF,COD_REGIAO,NOME_REGIAO,Grau de Instrucao,...,COD_CBO_GRUPO,DESC_CBO_GRUPO,COD_CBO_PSUB,DESC_CBO_PSUB,COD_CBO_SUBGRUPO,DESC_CBO_SUBGRUPO,COD_CBO_FAMILIA,DESC_CBO_FAMILIA,COD_CBO_OCUP,DESC_CBO_OCUP
0,012023,Homem,de 50 a 64 anos,Demitidos,13,AM,Amazonas,1,Norte,Superior completo,...,5,"TRABALHADORES DOS SERVIÇOS, VENDEDORES DO COMÉ...",51,TRABALHADORES DOS SERVIÇOS ...,517,TRABALHADORES NOS SERVIÇOS DE PROTEÇÃO E SEGUR...,5173,Vigilantes e guardas de segurança,517330,Vigilante
1,012023,Mulher,de 40 a 49 anos,Demitidos,13,AM,Amazonas,1,Norte,Médio completo,...,1,"MEMBROS SUPERIORES DO PODER PÚBLICO, DIRIGENTE...",14,GERENTES ...,142,GERENTES DE ÁREAS DE APOIO ...,1424,Gerentes de suprimentos e afins,142415,Gerente de almoxarifado
2,012023,Homem,de 25 a 29 anos,Admissão,21,MA,Maranhão,2,Nordeste,Superior completo,...,1,"MEMBROS SUPERIORES DO PODER PÚBLICO, DIRIGENTE...",14,GERENTES ...,142,GERENTES DE ÁREAS DE APOIO ...,1423,"Gerentes de comercialização, marketing e comun...",142345,Profissional de relações institucionais e gove...
3,012023,Homem,de 40 a 49 anos,Admissão,21,MA,Maranhão,2,Nordeste,Superior completo,...,2,PROFISSIONAIS DAS CIÊNCIAS E DAS ARTES ...,21,"PROFISSIONAIS DAS CIÊNCIAS EXATAS, FÍSICAS E D...",214,"ENGENHEIROS, ARQUITETOS E AFINS ...",2142,Engenheiros civis e afins,214205,Engenheiro civil
4,012023,Homem,de 30 a 39 anos,Admissão,21,MA,Maranhão,2,Nordeste,Médio completo,...,3,TÉCNICOS DE NIVEL MÉDIO ...,31,"TÉCNICOS DE NÍVEL MÉDIO DAS CIÊNCIAS FÍSICAS, ...",311,TÉCNICO EM CIÊNCIAS FÍSICAS E QUÍMICAS ...,3111,Técnicos químicos,311105,Técnico químico
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41852,012023,Homem,de 25 a 29 anos,Admissão,42,SC,Santa Catarina,4,Sul,Médio incompleto,...,7,TRABALHADORES DA PRODUÇÃO DE BENS E SERVIÇOS I...,78,TRABALHADORES DE FUNÇÕES TRANSVERSAIS ...,782,CONDUTORES DE VEÍCULOS E OPERADORES DE EQUIPAM...,7827,Trabalhadores aquaviários,782705,Marinheiro de convés (marítimo e fluviário)
41853,012023,Homem,de 50 a 64 anos,Admissão,42,SC,Santa Catarina,4,Sul,Médio completo,...,7,TRABALHADORES DA PRODUÇÃO DE BENS E SERVIÇOS I...,78,TRABALHADORES DE FUNÇÕES TRANSVERSAIS ...,782,CONDUTORES DE VEÍCULOS E OPERADORES DE EQUIPAM...,7827,Trabalhadores aquaviários,782705,Marinheiro de convés (marítimo e fluviário)
41854,012023,Homem,de 40 a 49 anos,Admissão,42,SC,Santa Catarina,4,Sul,Médio completo,...,7,TRABALHADORES DA PRODUÇÃO DE BENS E SERVIÇOS I...,78,TRABALHADORES DE FUNÇÕES TRANSVERSAIS ...,782,CONDUTORES DE VEÍCULOS E OPERADORES DE EQUIPAM...,7827,Trabalhadores aquaviários,782710,Marinheiro de máquinas
41855,012023,Homem,de 25 a 29 anos,Admissão,42,SC,Santa Catarina,4,Sul,Médio completo,...,7,TRABALHADORES DA PRODUÇÃO DE BENS E SERVIÇOS I...,78,TRABALHADORES DE FUNÇÕES TRANSVERSAIS ...,782,CONDUTORES DE VEÍCULOS E OPERADORES DE EQUIPAM...,7827,Trabalhadores aquaviários,782720,Moço de máquinas (marítimo e fluviário)


### PARA BAIXO É SOMENTE ANÁLISES ####

In [22]:
#colunas
cagedmov.columns

Index(['competenciamov', 'regiao', 'uf', 'municipio', 'secao', 'subclasse',
       'saldomovimentacao', 'cbo2002ocupacao', 'categoria', 'graudeinstrucao',
       'idade', 'horascontratuais', 'racacor', 'sexo', 'tipoempregador',
       'tipoestabelecimento', 'tipomovimentacao', 'tipodedeficiencia',
       'indtrabintermitente', 'indtrabparcial', 'salario', 'tamestabjan',
       'indicadoraprendiz', 'origemdainformacao', 'competenciadec',
       'indicadordeforadoprazo', 'unidadesalariocodigo', 'valorsalariofixo',
       'mesano_referencia'],
      dtype='object')

In [51]:
# pegando unico ano e vendo o tipo
cagedmov.competenciadec.unique()

array([202303], dtype=int64)

In [24]:
#total de demitidos
total_desl = sum(cagedmov['saldomovimentacao'] == -1)
total_desl

1973247

In [25]:
#total de admissões
total_admit = sum(cagedmov['saldomovimentacao'] == 1)
total_admit

2168418

In [26]:
# saldo de admissão - demissão
saldo = total_admit - total_desl
saldo

195171

In [32]:
#valores unicos por escolaridade
cagedmov['graudeinstrucao'].unique()

array([ 5,  7,  1,  8,  4,  9,  6,  2, 80, 10,  3, 11, 99], dtype=int64)

In [30]:
# total de analfabetos
total_grau_4 = sum(cagedmov['graudeinstrucao'] == 1)
total_grau_4

17915

In [31]:
# buscando valores de demissão e admissão
cagedmov['saldomovimentacao'].unique()

array([ 1, -1], dtype=int64)

In [35]:
# verificando as informações do dataframe
cagedfor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67160 entries, 0 to 67159
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   competenciamov          67160 non-null  int64 
 1   regiao                  67160 non-null  int64 
 2   uf                      67160 non-null  int64 
 3   municipio               67160 non-null  int64 
 4   secao                   67160 non-null  object
 5   subclasse               67160 non-null  int64 
 6   saldomovimentacao       67160 non-null  int64 
 7   cbo2002ocupacao         67160 non-null  int64 
 8   categoria               67160 non-null  int64 
 9   graudeinstrucao         67160 non-null  int64 
 10  idade                   67160 non-null  object
 11  horascontratuais        67160 non-null  object
 12  racacor                 67160 non-null  int64 
 13  sexo                    67160 non-null  int64 
 14  tipoempregador          67160 non-null  int64 
 15  ti

In [31]:
# Identificar linhas com valores nulos
linhas_com_nulos = cagedmov[cagedmov.isnull().sum(axis=1) > 0]
print("\nLinhas com valores nulos:")
print(linhas_com_nulos)


Linhas com valores nulos:
Empty DataFrame
Columns: [competenciamov, regiao, uf, municipio, secao, subclasse, saldomovimentacao, cbo2002ocupacao, categoria, graudeinstrucao, idade, horascontratuais, racacor, sexo, tipoempregador, tipoestabelecimento, tipomovimentacao, tipodedeficiencia, indtrabintermitente, indtrabparcial, salario, tamestabjan, indicadoraprendiz, origemdainformacao, competenciadec, indicadordeforadoprazo, unidadesalariocodigo, valorsalariofixo, mesano_referencia]
Index: []

[0 rows x 29 columns]


In [32]:
# Contar o número de linhas com valores nulos
total_linhas_nulos = cagedxc.isnull().any(axis=1).sum()
print("Total de linhas com valores nulos:", total_linhas_nulos)

Total de linhas com valores nulos: 0


In [24]:
# buscando total de admissão por grau de instrução
adm_graus = {}  # Cria um dicionário vazio para armazenar o total de cada admitido por grau de instrução
for i in range(1, 12):  # Loop para percorrer os valores de 1 a 11 (inclusive)
    adm_graus[i] = sum((cagedmov['graudeinstrucao'] == i) & (cagedmov['saldomovimentacao'] == 1))
    # Calcula a soma das condições:
    # - Verifica se o valor em 'graudeinstrucao' é igual a 'i'
    # - Verifica se o valor em 'saldomovimentacao' é igual a 1
    # Soma os resultados das condições e atribui ao dicionário 'agraus' usando 'i' como chave

adm_graus[80] = sum((cagedmov['graudeinstrucao'] == 80) & (cagedmov['saldomovimentacao'] == 1))
# Calcula a soma das condições para o grau 80 e atribui ao dicionário 'agraus' com chave 80

adm_graus[99] = sum((cagedmov['graudeinstrucao'] == 99) & (cagedmov['saldomovimentacao'] == 1))
# Calcula a soma das condições para o grau 99 e atribui ao dicionário 'agraus' com chave 99

In [25]:
# total sem soma de admissão por instrução
print(f'Lista de Admissão\n\n'
      f'Analfabeto: {adm_graus[1]}\n'
      f'Até 5ª Incompleto: {adm_graus[2]}\n'
      f'5ª Completo Fundamental: {adm_graus[3]}\n'
      f'6ª a 9ª Fundamental: {adm_graus[4]}\n'
      f'Fundamental Completo: {adm_graus[5]}\n'
      f'Médio Incompleto: {adm_graus[6]}\n'
      f'Médio Completo: {adm_graus[7]}\n'
      f'Superior Incompleto: {adm_graus[8]}\n'
      f'Superior Completo: {adm_graus[9]}\n'
      f'MESTRADO: {adm_graus[10]}\n'
      f'DOUTORADO: {adm_graus[11]}\n'
      f'Pós-Graduação completa: {adm_graus[80]}\n'
      f'Não Identificado: {adm_graus[99]}')

Lista de Admissão

Analfabeto: 8810
Até 5ª Incompleto: 37887
5ª Completo Fundamental: 28858
6ª a 9ª Fundamental: 94946
Fundamental Completo: 154070
Médio Incompleto: 162374
Médio Completo: 1392551
Superior Incompleto: 82439
Superior Completo: 179340
MESTRADO: 4213
DOUTORADO: 1606
Pós-Graduação completa: 21324
Não Identificado: 0


In [26]:
# total com soma de admissão por instrução
print(f'Lista de Admissão\n\n'
      f'Analfabeto: {adm_graus[1]}\n'
      f'Fundamental Incompleto: {adm_graus[2] + adm_graus[3] + adm_graus[4]}\n'
      f'Fundamental Completo: {adm_graus[5]}\n'
      f'Médio Incompleto: {adm_graus[6]}\n'
      f'Médio Completo: {adm_graus[7]}\n'
      f'Superior Incompleto: {adm_graus[8]}\n'
      f'Superior Completo: {adm_graus[9] + adm_graus[10] + adm_graus[11] + adm_graus[80]}\n'
      f'Não Identificado: {adm_graus[99]}')

Lista de Admissão

Analfabeto: 8810
Fundamental Incompleto: 161691
Fundamental Completo: 154070
Médio Incompleto: 162374
Médio Completo: 1392551
Superior Incompleto: 82439
Superior Completo: 206483
Não Identificado: 0


In [27]:
# buscando total de demissão por grau de instrução
dmt_graus = {}  # Cria um dicionário vazio para armazenar o total de cada admitido por grau de instrução
for i in range(1, 12):  # Loop para percorrer os valores de 1 a 11 (inclusive)
    dmt_graus[i] = sum((cagedmov['graudeinstrucao'] == i) & (cagedmov['saldomovimentacao'] == -1))
    # Calcula a soma das condições:
    # - Verifica se o valor em 'graudeinstrucao' é igual a 'i'
    # - Verifica se o valor em 'saldomovimentacao' é igual a 1
    # Soma os resultados das condições e atribui ao dicionário 'agraus' usando 'i' como chave

dmt_graus[80] = sum((cagedmov['graudeinstrucao'] == 80) & (cagedmov['saldomovimentacao'] == -1))
# Calcula a soma das condições para o grau 80 e atribui ao dicionário 'agraus' com chave 80

dmt_graus[99] = sum((cagedmov['graudeinstrucao'] == 99) & (cagedmov['saldomovimentacao'] == -1))
# Calcula a soma das condições para o grau 99 e atribui ao dicionário 'agraus' com chave 99

In [28]:
# total sem soma de demissão por instrução
print(f'Lista de Admissão\n\n'
      f'Analfabeto: {dmt_graus[1]}\n'
      f'Até 5ª Incompleto: {dmt_graus[2]}\n'
      f'5ª Completo Fundamental: {dmt_graus[3]}\n'
      f'6ª a 9ª Fundamental: {dmt_graus[4]}\n'
      f'Fundamental Completo: {dmt_graus[5]}\n'
      f'Médio Incompleto: {dmt_graus[6]}\n'
      f'Médio Completo: {dmt_graus[7]}\n'
      f'Superior Incompleto: {dmt_graus[8]}\n'
      f'Superior Completo: {dmt_graus[9]}\n'
      f'MESTRADO: {dmt_graus[10]}\n'
      f'DOUTORADO: {dmt_graus[11]}\n'
      f'Pós-Graduação completa: {dmt_graus[80]}\n'
      f'Não Identificado: {dmt_graus[99]}')

Lista de Admissão

Analfabeto: 9105
Até 5ª Incompleto: 41821
5ª Completo Fundamental: 32299
6ª a 9ª Fundamental: 90891
Fundamental Completo: 147883
Médio Incompleto: 144981
Médio Completo: 1232241
Superior Incompleto: 81832
Superior Completo: 167916
MESTRADO: 3225
DOUTORADO: 1150
Pós-Graduação completa: 19902
Não Identificado: 1


In [29]:
# total com soma de demissão por instrução
print(f'Lista de Admissão\n\n'
      f'Analfabeto: {dmt_graus[1]}\n'
      f'Fundamental Incompleto: {dmt_graus[2] + dmt_graus[3] + dmt_graus[4]}\n'
      f'Fundamental Completo: {dmt_graus[5]}\n'
      f'Médio Incompleto: {dmt_graus[6]}\n'
      f'Médio Completo: {dmt_graus[7]}\n'
      f'Superior Incompleto: {dmt_graus[8]}\n'
      f'Superior Completo: {dmt_graus[9] + dmt_graus[10] + dmt_graus[11] + dmt_graus[80]}\n'
      f'Não Identificado: {dmt_graus[99]}')

Lista de Admissão

Analfabeto: 9105
Fundamental Incompleto: 165011
Fundamental Completo: 147883
Médio Incompleto: 144981
Médio Completo: 1232241
Superior Incompleto: 81832
Superior Completo: 192193
Não Identificado: 1


In [30]:
#saldo por grau de instrução
a1 = adm_graus[1] - dmt_graus[1]
a2 = (adm_graus[2] + adm_graus[3] + adm_graus[4]) - (dmt_graus[2] + dmt_graus[3] + dmt_graus[4])
a3 = adm_graus[5] - dmt_graus[5]
a4 = adm_graus[6] - dmt_graus[6]
a5 = adm_graus[7] - dmt_graus[7]
a6 = adm_graus[8] - dmt_graus[8]
a7 = (adm_graus[9] + adm_graus[10] + adm_graus[11] + adm_graus[80]) - (dmt_graus[9] + dmt_graus[10] + dmt_graus[11] + dmt_graus[80])
a8 = adm_graus[99] - dmt_graus[99]

print(f'Saldo\n\n'
      f'Analfabeto: {a1}\n'
      f'Fundamental Incompleto: {a2}\n'
      f'Fundamental Completo: {a3}\n'
      f'Médio Incompleto: {a4}\n'
      f'Médio Completo: {a5}\n'
      f'Superior Incompleto: {a6}\n'
      f'Superior Completo: {a7}\n'
      f'Não Identificado: {a8}')

Saldo

Analfabeto: -295
Fundamental Incompleto: -3320
Fundamental Completo: 6187
Médio Incompleto: 17393
Médio Completo: 160310
Superior Incompleto: 607
Superior Completo: 14290
Não Identificado: -1


In [ ]:
# if ano < 2020: 
#     url = f"ftp://ftp.mtps.gov.br/pdet/microdados/CAGED/{ano}/CAGEDEST_{mes}{ano}.7z"
#     wget.download(url, 'caged.7z')
#     archive = SevenZipFile('caged.7z', mode = 'r')
#     archive.extractall()
#     print('microdados baixados e extraídos com sucesso')
    
#     data = pd.read_csv(f'CAGEDEST_{mes}{ano}.txt', sep = ';', encoding = 'iso-8859-1')
#     print(f'dataframe {mes}-{ano} montado com sucesso')
    
#     archive.close()
#     remove('caged.7z') 
#     del url
#     print('ftp desconectado e arquivos removidos')
    
# elif ano >= 2020:
#     url = f'ftp://ftp.mtps.gov.br/pdet/microdados/NOVO CAGED/{ano}/{ano}{mes}/CAGEDMOV{ano}{mes}.7z'
#     try:
#         wget.download(url, 'caged.7z')
#     except:
#         print('Os microdados do mês selecionado ainda não estão disponíveis')
        
#     archive = SevenZipFile('caged.7z', mode = 'r')
#     archive.extractall()
#     print('microdados baixados com sucesso.')
    
#     data = pd.read_csv(f'CAGEDMOV{ano}{mes}.txt', sep = ';', encoding = 'UTF-8')
#     print(f'dataframe {mes}-{ano} montado com sucesso')
    
#     archive.close()
#     remove('caged.7z')
#     del url
#     print('ftp desconectado e arquivos removidos')

# todos os dados com grau de instrução até 80 de admitidos
#cagedmov.loc[(cagedmov['graudeinstrucao'] == 80) & (cagedmov['saldomovimentacao'] == 1)]